In [ ]:
import random

# Programação funcional em Python
*Sintam-se a vontade para pular esta parte. Achei importante incluir porque talvez alguém não conheça esses recursos da linguagem*.

Programação funcional é um paradigma de programação com foco em funções puras, imutabilidade de dados, recursão e funções de alta ordem. Apesar de Python não ser uma linguagem funcional, este é um paradigma de que eu gosto e o código abaixo segue alguns destes princípios. Vejamos alguns recursos de programação funcional disponíveis em Python que foram utilizados:

### List comprehensions
Uma maneira concisa de criar listas em Python, aplicando uma expressão a cada item em um iterável (por exemplo, uma lista) e coletando os resultados. Consistem em colchetes `[]` que contêm uma expressão seguida de um loop for (e, opcionalmente, condições if) para filtrar ou modificar elementos do iterável.

    [x*x for x in range(1,6)] 
    # -> [1,4,9,16,25]
    
    [x for x in [32,1,6,48,2,75] if x % 2 == 1]
    # -> [1, 75]
    
### Funções de alta ordem
Em Python, as funções são consideradas first-class citizens, o que significa que podem ser tratadas como qualquer outro tipo de dado, como inteiros ou strings. Uma função de alta ordem é uma função que recebe uma ou mais funções como argumentos ou retorna uma função como seu resultado

    def opera(f, a, b):
        return f(a, b)
    
    def soma(a, b):
        return a + b
        
    opera(soma, 1, 2)
    # -> 3
    
    def soma3(a):
        return soma(3, a)
    
    soma3(2)
    # -> 5

### Expressões lambda
Uma maneira simples de definir uma função em uma linha. Pode ser atribuida a uma variável, ou ser passada anonimamente para uma função de alta ordem.

    quadrado = lambda x: x*x
    
    quadrado(7)
    # -> 49

In [ ]:
# Retorna uma copia da lista x embaralhada, sem modificar a lista original
def imut_shuffle(x):
    return random.sample(x, k=len(x))

In [ ]:
# Retorna uma List contendo todos os caracteres imprimiveis da tabela ASCII
def get_printables():
    return "".join([chr(i) for i in range (32,127)])

# O Rotor

O rotor é a peça móvel do Enigma. Ele contém 26 contatos, ou quantos forem suficientes para representar todos os caracteres que se deseja criptografar (no nosso caso serão todos os caracteres imprimíveis da tabela ASCII, isso é 95 caracteres). Internamente, o rotor "embaralha" as entradas, conectando uma letra na entrada para uma letra diferente na saída.

Usualmente, o Enigma conta três rotores conectados em sequência (em nosso projeto poderemos ter tantos rotores quanto forem necessários). Após a inserção de um caracter qualquer, o primeiro rotor avança uma posição, criando novas conexões entre as letras, ou seja, uma mesma letra de entrada poderá ter múltiplas representações distintas na mensagem criptografada. Isso torna o Enigma uma máquina criptográfica bastante robusta.

Vejamos o funcionamento de um rotor. Suponha que temos um rotor com as conexões abaixo:

| A | B | C | D | E | F | G | H | I | J | K | L | M | N | O | P | Q | R | S | T | U | V | W | X | Y | Z | 
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| D | M | T | W | S | I | L | R | U | Y | Q | N | K | F | E | J | C | A | Z | B | P | G | X | O | H | V |

Com esta configuração, se a nossa mensagem começa com a letra 'U', após passar pelo primeiro rotor, se transformará em 'P'. Em seguida o rotor avançará uma posição e as novas conexões serão:

| A | B | C | D | E | F | G | H | I | J | K | L | M | N | O | P | Q | R | S | T | U | V | W | X | Y | Z | 
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| V | D | M | T | W | S | I | L | R | U | Y | Q | N | K | F | E | J | C | A | Z | B | P | G | X | O | H |

Se a segunda letra da mensagem também for 'U', dessa vez se transformará em 'B'.

# A Simetria do Enigma

O método de criptografia do Enigma é "simétrico", isto  é, dado um algoritmo de criptografia `Enc`, uma chave `k` e uma mensagem cifrada `c` gerada a partir de uma mensagem `m`, `c = Enc(k, m)` então `m = Enc(k, c)`. Ou seja, podemos usar o mesmo algoritmo e chave para cifrar uma mensagem e decifrar uma mensagem cifrada.

A simetria é garantida pelo uso do refletor. Originalmente, em uma máquina de Enigma eletromecânica, o sinal partia do teclado onde o operador digitava a mensagem que se queria criptografar, passava por um conjunto de rotores, depois pelo refletor e por fim **passava novamente pelos rotores em ordem inversa**.

Suponhamos que em uma Máquina com três rotores com uma dada configuração inicial, queremos cifrar a letra 'A' e o sinal foi: 

| Ida     |         |         |          | Volta   |         |         | 
|---------|---------|---------|----------|---------|---------|---------| 
| 1 Rotor | 2 Rotor | 3 Rotor | Refletor | 3 Rotor | 2 Rotor | 1 Rotor | 
| A       | F       | A       | G        | Y       | L       | C       |

Então **na mesma Máquina, com os mesmos rotores e configuração inicial,** se quisermos cifrar a letra 'C', o sinal será: 

| Ida     |         |         |          | Volta   |         |         | 
|---------|---------|---------|----------|---------|---------|---------| 
| 1 Rotor | 2 Rotor | 3 Rotor | Refletor | 3 Rotor | 2 Rotor | 1 Rotor | 
| C       | L       | Y       | G        | A       | F       | A       |


Essa escolha de design permitiu a simplificação da operação, já que apenas um algoritmo e chave são necessários tanto na cifragem quanto na decifragem, mas também introduziu uma vulnerabilidade que foi utilizada para quebrar o Enigma: nenhuma letra poderia cifrar ela própria, pois para isso o sinal teria que passar pelo mesmo contato em dois sentidos diferentes, o que não é possível. Nossa implementação não terá essa vulnerabilidade, pois podemos simular este comportamento em software.

Mais detalhes sobre o Enigma em: https://pt.wikipedia.org/wiki/Enigma_(m%C3%A1quina)

In [ ]:
def create_rotor():
    rotor = imut_shuffle(get_printables())
    l = len(rotor)
    
    printables = get_printables()
    
    signal_up = lambda c, rotation: rotor[(printables.index(c)+rotation) % l]
    signal_down = lambda c, rotation: printables[(rotor.index(c)-rotation) % l]
    return lambda c, rotation, up=True: signal_up(c, rotation) if up else signal_down(c, rotation)

In [ ]:
# Retorna a string que o rotor usara para criptografar
def get_rotor_string(rotor):
    return "".join( [rotor(c, 0) for c in get_printables()] )

In [ ]:
# Imprime o relatorio de um teste que retorna a lista de testes passados e falhados, nesta ordem
# Devolve `True` se todos os casos passarem
def TEST_report(test):
    
    passed, failed = test()
    
    print(f"Passed: {len(passed)}\nFailed: {len(failed)}")
    
    return failed == []

In [ ]:
def TEST_create_rotor():
    r = create_rotor()
    
    passed = []
    failed = []
    
    printables = get_printables()
    
    for i in range(len(printables)):
        if ("".join( [r( r( c, i, up=False), i ) for c in printables] )) == printables:
            passed.append(i)
        else:
            failed.append(i)
        
    return passed, failed

In [ ]:
TEST_report(TEST_create_rotor)

In [ ]:
# Testa se, dado uma seed ao gerador aleatorio, o rotor gerado sera sempre o mesmo
# Meio obvio, mas nao custa testar :P
def TEST_determine_rotor():
    passed = []
    failed = []
    
    random.seed(42)
    reference = get_rotor_string( create_rotor() )
    
    for _ in range(100):
        random.seed(42)
        
        rot_str = get_rotor_string( create_rotor() )
        
        if rot_str == reference:
            passed.append(True)
        else:
            failed.append(False)
    
    return passed, failed

In [ ]:
TEST_report(TEST_determine_rotor)

In [ ]:
# Em media, apenas um caracter do rotor gera ele mesmo
def TEST_encrypt_same_caracter():

    same_index = 0
    
    for _ in range(100):
        r = create_rotor()

        rot_str = get_rotor_string(r)
        printables = get_printables()
        
        same_index += len([c for c in printables if rot_str.index(c) == printables.index(c)])
        
    return same_index / 100

# O Refletor

O refletor é uma peça fixa do enigma, indispensável para garantir a simetria anteriormente mencionada. Ele é construído de forma diferente do rotor: é necessário garantir que se `a` se tranforma em `b` ao passar pelo refletor, então também `b` se transforma em `a` ao passar pelo refletor. Note que no rotor isso não necessariamente ocorre (de fato, no rotor de exemplo discutido anteriormente **A** se transforma em **D**, mas **D**, se tranforma em **W**). No Enigma eletromecânico, havia ainda outra restrição, que é `a` deve ser diferente de `b`. Na nossa implementação não teremos essa restrição.

Vejamos um refletor de exemplo:

| A | B | C | D | E | F | G | H | I | J | K | L | M | N | O | P | Q | R | S | T | U | V | W | X | Y | Z | 
|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|
| Q | Y | H | O | G | N | E | C | V | P | U | Z | T | F | D | J | A | X | W | M | K | I | S | R | B | L |

Aqui, **A** se transforma em **Q**, e também **Q** se transforma em **A**. Este sempre será o caso, já que o refletor é fixo.
 

In [ ]:
# Cria um refletor conforme discutido acima. 
# O algoritmo seleciona dois caracteres aleatoriamente da lista de imprimiveis e os troca de lugar.
# Uma vez posicionado um caracter nao trocara mais de lugar
# Como temos um numero impar de caracteres, exatamente um caracter ficara sem par e se cifrara ele proprio

# Podemos pensar em um outro algoritmo que permita que outros caracteres tambem cifrem eles proprios

def create_reflector(missing_test=False):
    
    printables = [*get_printables()]
    mut_printables = [*get_printables()]
    
    l = len(printables)
    
    reflector = [''] * l
    
    for i in range(l//2):
        
        p1 = random.choice(mut_printables)
        mut_printables.remove(p1)
        
        p2 = random.choice(mut_printables)
        mut_printables.remove(p2)
        
        reflector[printables.index(p1)] = p2
        reflector[printables.index(p2)] = p1
        
    
    reflector[reflector.index('')] = mut_printables[0]
    
    if missing_test:
        return "".join(reflector), mut_printables
    
    return lambda c: printables[reflector.index(c)]

In [ ]:
def get_reflector_string(reflector):
    return "".join( [reflector(c) for c in get_printables()] )

In [ ]:
# Este teste mostra que no maximo um caractere e 'perdido' na criacao do refletor
# Isso acaba acontecendo porque o numero de caracteres imprimiveis e impar
def TEST_check_for_missing():
    passed = []
    failed = []
    
    
    for _ in range(10000):
        ref, miss = create_reflector(missing_test=True)
        
        if len(miss) <= 1:
            passed.append(True)
        else:
            failed.append(False)
    
    return passed, failed

In [ ]:
TEST_report(TEST_check_for_missing)

In [ ]:
# Retorna uma funcao (Maquina) que utilizara um numero especificado de rotores para realizar a criptografia da mensagem
# Recebe `random_seed` para garantir reprodutibilidade
def create_encryptor(rot_count, random_seed):
    random.seed(random_seed)
    
    rotors = [create_rotor() for _ in range(rot_count)]
    reflector = create_reflector()
    
    def encryptor(c, state):
        new_c = c
        l = len(get_printables())
        
        rotor_states = []
        
        for _ in range(rot_count):
            rotor_states.append(state % l)
            state = state // l
        
        for r, s in zip(rotors, rotor_states):
            new_c = r(new_c, s)
        
        new_c = reflector(new_c)
        
        for r, s in zip(rotors[::-1], rotor_states[::-1]):
            new_c = r(new_c, s, up=False)
        
        return new_c
    
    return encryptor

In [ ]:
# Remove caracteres de controle, ex: '\n'
def sanitize(string):
    return "".join(c for c in string if 32 <= ord(c) < 127)

In [ ]:
# Recebe uma Maquina, uma mensagem e uma configuracao inicial
# Retorna a mensagem cifrada e a configuracao final
def encrypt(encryptor, msg, initial_state):
    encrypted_message = []
    state = initial_state
    
    sanitized = sanitize(msg)
    
    for c in sanitized:
        new_c = encryptor(c, state)
        state = state + 1
        
        encrypted_message.append(new_c)
        
    return "".join(encrypted_message)

In [ ]:
enc = create_encryptor(3, 163)

In [ ]:
message = """
Esta e uma mensagem multilinha.
Antes de ser cifrada, os caracteres de controle deverao ser removidos.
Esses caracteres nao estarao presentes quando decifrarmos a mensagem
"""

In [ ]:
initial_state = 1729

In [ ]:
cypher = encrypt(enc, message, initial_state)
cypher

In [ ]:
decypher = encrypt(enc, cypher, initial_state)
decypher